In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats 

In [2]:
gett = pd.read_csv('Netology_A_B-testing_HW_1.csv')
gett.head()

,Unnamed: 0,date,user_id,hour,app_opened,price_seen,order_made,surge,ride_completed,user_cancelled,age,os,city_center_order,order_class,distance,rfm
0,0,2020-05-21,867689,12,1,1,1,no surge,1,0,20,iOS,0,business,7.982135,low
1,1,2020-05-23,752172,5,1,1,1,no surge,1,0,37,Android,1,economy,2.908468,high
2,2,2020-05-20,486559,15,1,1,1,no surge,1,0,47,Android,0,comfort,7.224614,low
3,3,2020-05-19,304024,0,1,1,1,no surge,1,0,59,Android,1,economy,1.874349,low
4,4,2020-05-23,139420,0,1,1,1,no surge,1,0,19,Android,0,business,10.704778,medium


### Особенность - "Для устройств Android наблюдаем более высокие юзерские отмены"

In [3]:
gett_and = gett[gett['os'] == 'Android']['user_cancelled'].sample(n = 44000)
gett_and.value_counts()

0    38272
1     5728
Name: user_cancelled, dtype: int64

In [4]:
gett_ios = gett[gett['os'] == 'iOS']['user_cancelled'].sample(n = 44000)
gett_ios.value_counts()

0    40152
1     3848
Name: user_cancelled, dtype: int64

In [5]:
T, p = stats.ttest_ind(gett_and, gett_ios) 
T, p

(20.398680045088714, 2.8117680354652814e-92)

In [6]:
# пи-вэлью крайне мал, следовательно вероятность ошибиться, отвергнув Н0 (что различий нет), практически отсутствует. 
# Поэтому различия в выборках действительно есть! Т.е. у андроидов отмены встречаются чаще.


### Особенность - "В период Surge видим более низкую price-to-order конверсию"

In [7]:
gett_sur = gett[(gett['surge'] == 'surge') & gett['price_seen'] == 1]['order_made'].sample(n = 25000)
gett_sur.value_counts()

1    18085
0     6915
Name: order_made, dtype: int64

In [8]:
gett_nosur = gett[(gett['surge'] == 'no surge') & gett['price_seen'] == 1]['order_made'].sample(n = 25000)
gett_nosur.value_counts()

1    21472
0     3528
Name: order_made, dtype: int64

In [9]:
T2, p2 = stats.ttest_ind(gett_sur, gett_nosur) 
T2, p2

(-37.790511937481725, 3.6630404390410485e-308)

In [10]:
# пи-вэлью крайне мал, следовательно вероятность ошибиться, отвергнув Н0 (что различий нет), практически отсутствует. 
# Поэтому различия в выборках действительно есть! Т.е. с наценкой конверсия в заказ менее вероятна, чем без неё.

### Делим начальную выборку на тестовую и контрольную

In [13]:
gett_test = gett.sample(frac=0.5, random_state = 1)
len(gett_test)

50750

In [23]:
gett_control = gett[~gett.index.isin(gett_test.index)]
len(gett_control)

50750

In [22]:
gett_test[gett_test['user_id'] == 519899]

,Unnamed: 0,date,user_id,hour,app_opened,price_seen,order_made,surge,ride_completed,user_cancelled,age,os,city_center_order,order_class,distance,rfm
64345,64345,2020-05-23,519899,4,1,1,1,no surge,1,0,32,iOS,1,economy,10.436720,medium
12073,12073,2020-05-20,519899,19,1,1,1,surge,1,0,45,Android,0,comfort,3.031510,low
56813,56813,2020-05-18,519899,7,1,1,1,surge,1,0,47,iOS,1,comfort,9.921212,medium
53963,53963,2020-05-20,519899,12,1,1,0,no surge,0,0,30,Android,1,comfort,2.785819,low


Теперь проверим однородность этих выборок для будущего теста Гипотезы 1 (Если предусмотреть надбавку только начиная с определённой дистанции (которую сделаем чуть меньше средней дальности поездки, чтобы большинство попадало под неё) при небольшом увеличении этой надбавки, то это обеспечит более высокую конверсию (~на 10%))

Чтобы увидеть эффект фичи, нам нужно, чтобы изначально у двух выборок и так была одинаковая конверсия, т.е. order_made

In [25]:
T3, p3 = stats.ttest_ind(gett_test.order_made, gett_control.order_made) 
T3, p3

(-0.09914132775912708, 0.9210262546878499)

In [26]:
### огромнейший пи-велью --- выборки почти идентичны.

Теперь проверим однородность этих выборок для будущего теста Гипотезы 2 (При введение подарочного промо кода на следующую поездку или в магазинах-партнерах по завершении заказа для пользователей андроидов можно повысить процент успешно завершённых поездок)

Для этого смотрим на столбец user_cancelled

In [27]:
T4, p4 = stats.ttest_ind(gett_test.user_cancelled, gett_control.user_cancelled) 
T4, p4

(-0.1898286877943655, 0.8494437546110172)

In [ ]:
### огромнейший пи-велью --- выборки почти идентичны.

ВЫВОД: можно испытывать новые фичи на половине клиентов, а потом сравним их с другой, которая была без изменений.